## Introduction

Patterns are a special syntax in Rust for matching against the structure of types, both complex and simple. Using patterns in conjunction with `match` expressions and other constructs gives you more control over a program’s control flow. A pattern consists of some combination of the following:

- Literals
- Destructured arrays, enums, structs, or tuples
- Variables
- Wildcards
- Placeholders

## All the Places Patterns Can Be Used

### `match` Arms

```rust
match VALUE {
    PATTERN => EXPRESSION,
    PATTERN => EXPRESSION,
    PATTERN => EXPRESSION,
}
```

Example: 
```rust
match x {
    None => None,
    Some(i) => Some(i + 1),
}
```

<span style="color:lightgreen">*One requirement for `match` expressions is that they need to be exhaustive in the sense that all possibilities for the value in the `match` expression must be accounted for.*</span>

### Conditional `if let` Expressions

We can mix `if let`, `else if`, `else if let`, and `else`. For example, below we made a code to tell a story:  
"If the user specifies a favorite color, that color is used as the background. If no favorite color is specified and today is Tuesday, the background color is green. Otherwise, if the user specifies their age as a string and we can parse it as a number successfully, the color is either purple or orange depending on the value of the number. If none of these conditions apply, the background color is blue."

In [3]:
fn main() {
    let favorite_color: Option<&str> = None;
    let is_tuesday = false;
    let age: Result<u8, _> = "34".parse();

    if let Some(color) = favorite_color {
        println!("Using your favorite color, {color}, as the background");
    } else if is_tuesday {
        println!("Tuesday is green day!");
    } else if let Ok(age) = age {
        if age > 30 {
            println!("Using purple as the background color");
        } else {
            println!("Using orange as the background color");
        }
    } else {
        println!("Using blue as the background color");
    }
}

main()

Using purple as the background color


()

This conditional structure lets us support complex requirements. <span style="color:orange">*The downside of using `if let` expressions is that the compiler doesn’t check for exhaustiveness, whereas with `match` expressions it does*</span>

### `while let` Conditional Loops

Similar in construction to if let, the while let conditional loop allows a while loop to run for as long as a pattern continues to match.

In [2]:
fn main() {
    let mut stack = Vec::new();

    stack.push(1);
    stack.push(2);
    stack.push(3);

    while let Some(top) = stack.pop() {
        println!("{}", top);
    }
}
main()

3
2
1


()

### `for` Loops

In a `for` loop, the value that directly follows the keyword `for` is a pattern

In [4]:
let v = vec!['a', 'b', 'c'];

for (index, value) in v.iter().enumerate() {
    println!("{} is at index {}", value, index);
}


a is at index 0
b is at index 1
c is at index 2


()

### `let` Statements

```rust
let PATTERN = EXPRESSION;
```

For example:

In [6]:
let (x, y, z) = (1, 2, 3);  // this works

In [7]:
let (x, y) = (1, 2, 3); // this does not work

Error: mismatched types

### Function Parameters

Function parameters can also be patterns, for example

In [8]:
fn print_coordinates(&(x, y): &(i32, i32)) {
    println!("Current location: ({}, {})", x, y);
}

fn main() {
    let point = (3, 5);
    print_coordinates(&point);
}

main()

Current location: (3, 5)


()

The values `&(3, 5)` match the pattern `&(x, y)`, so `x` is the value `3` and `y` is the value `5`

## Refutability: Whether a Pattern Might Fail to Match

Patterns come in two forms: refutable and irrefutable
- Patterns that will match for any possible value passed are *irrefutable*. An example would be `x` in the statement `let x = 5`; because `x` matches anything and therefore cannot fail to match.
- Patterns that can fail to match for some possible value are *refutable*. An example would be `Some(x)` in the expression `if let Some(x) = a_value` because if the value in the `a_value` variable is `None` rather than `Some`, the `Some(x)` pattern will not match.

Function parameters, `let` statements, and `for` loops can only accept irrefutable patterns, because the program cannot do anything meaningful when values don’t match. The `if let` and `while let` expressions accept refutable and irrefutable patterns, but the compiler warns against irrefutable patterns because by definition they’re intended to handle possible failure: the functionality of a conditional is in its ability to perform differently depending on success or failure.

In general, you shouldn’t have to worry about the distinction between refutable and irrefutable patterns; however, you do need to be familiar with the concept of refutability so you can respond when you see it in an error message.  
If we have a refutable pattern where an irrefutable pattern is needed, we can fix it by changing the code that uses the pattern: instead of using `let`, we can use `if let`

## Pattern Syntax

### Matching Literals

```rust
    let x = 1;

    match x {
        1 => println!("one"),
        2 => println!("two"),
        3 => println!("three"),
        _ => println!("anything"),
    }
```

### Matching Named Variables

<span style="color:lightgreen">*Because `match` starts a new scope, variables declared as part of a pattern inside the `match` expression will shadow those with the same name outside the match construct, as is the case with all variables.*</span>



```rust
    let x = Some(5);
    let y = 10;

    match x {
        Some(50) => println!("Got 50"),
        Some(y) => println!("Matched, y = {y}"),  // this is a new y variable, not the y we declared at the beginning with the value 10. This new y binding will match any value inside a Some, which is what we have in x. Therefore, this new y binds to the inner value of the Some in x.
        _ => println!("Default case, x = {:?}", x),
    }

    println!("at the end: x = {:?}, y = {y}", x);
```

### Multiple Patterns

In `match` expressions, you can `match` multiple patterns using the `|` syntax, which is the pattern or operator. 

In [9]:
fn main() {
    let x = 1;

    match x {
        1 | 2 => println!("one or two"),
        3 => println!("three"),
        _ => println!("anything"),
    }
}

main()

one or two


()

### Matching Ranges of Values with `..=`

The `..=` syntax allows us to match to an inclusive range of values.

In [10]:
fn main() {
    let x = 5;

    match x {
        1..=5 => println!("one through five"),
        _ => println!("something else"),
    }
}

main()

one through five


()

In [12]:
fn main() {
    let x = 'z';

    match x {
        'a'..='j' => println!("early ASCII letter"),
        'k'..='z' => println!("late ASCII letter"),
        _ => println!("something else"),
    }
}

main()

late ASCII letter


()

### Destructuring to Break Apart Values

We can also use patterns to destructure structs, enums, and tuples to use different parts of these values. 

#### Destructuring Structs

In [14]:
struct Point {
    x: i32,
    y: i32,
}

fn main() {
    let p = Point { x: 0, y: 7 };

    let Point { x: a, y: b } = p;
    assert_eq!(0, a);
    assert_eq!(7, b);

    println!("a = {}, b = {}", a, b);
}

main()

a = 0, b = 7


()

We can write this in a shorter way with

In [16]:
struct Point {
    x: i32,
    y: i32,
}

fn main() {
    let p = Point { x: 0, y: 7 };

    let Point { x, y } = p;
    assert_eq!(0, x);
    assert_eq!(7, y);
    println!("x = {}, y = {}", x, y);
}

main()

x = 0, y = 7


()

We can also destructure and match literal values in one pattern

In [17]:
fn main() {
    let p = Point { x: 0, y: 7 };

    match p {
        Point { x, y: 0 } => println!("On the x axis at {x}"),
        Point { x: 0, y } => println!("On the y axis at {y}"),
        Point { x, y } => {
            println!("On neither axis: ({x}, {y})");
        }
    }
}

main()

On the y axis at 7


()

#### Destructuring Enums

Below, we destructure enum variants that hold different kinds of values

In [22]:
enum Message {
    Quit,
    Move { x: i32, y: i32 },
    Write(String),
    ChangeColor(i32, i32, i32),
}

fn main() {
    // let msg = Message::ChangeColor(0, 160, 255);
    let msg = Message::Move {x: 32, y: 11};

    match msg {
        Message::Quit => {
            println!("The Quit variant has no data to destructure.");
        }
        Message::Move { x, y } => {
            println!("Move in the x direction {x} and in the y direction {y}");
        }
        Message::Write(text) => {
            println!("Text message: {text}");
        }
        Message::ChangeColor(r, g, b) => {
            println!("Change the color to red {r}, green {g}, and blue {b}",)
        }
    }
}

main()

Move in the x direction 32 and in the y direction 11


()

#### Destructuring Nested Structs and Enums

In [23]:
enum Color {
    Rgb(i32, i32, i32),
    Hsv(i32, i32, i32),
}

enum Message {
    Quit,
    Move { x: i32, y: i32 },
    Write(String),
    ChangeColor(Color),
}

fn main() {
    let msg = Message::ChangeColor(Color::Hsv(0, 160, 255));

    match msg {
        Message::ChangeColor(Color::Rgb(r, g, b)) => {
            println!("Change color to red {r}, green {g}, and blue {b}");
        }
        Message::ChangeColor(Color::Hsv(h, s, v)) => {
            println!("Change color to hue {h}, saturation {s}, value {v}")
        }
        _ => (),
    }
}

main()

Change color to hue 0, saturation 160, value 255


()

#### Destructuring Structs and Tuples
We can mix, match, and nest destructuring patterns in even more complex ways

In [31]:
let ((feet, inches), Point { x, y }) = ((3, 10), Point {x: 3, y: -10 });

println!("feet = {}, x = {}, y = {}", feet, x, y);

feet = 3, x = 3, y = -10


### Ignoring Values in a Pattern

There are a few ways to ignore entire values or parts of values in a pattern:
- Using the `_` pattern (which you’ve seen),
- using the `_` pattern within another pattern,
- using a name that starts with an underscore,
- or using `..` to ignore remaining parts of a value. Let’s explore how and why to use each of these patterns.

#### Ignoring an Entire Value with `_`

In [32]:
fn foo(_: i32, y: i32) {
    println!("This code only uses the y parameter: {}", y);
}

fn main() {
    foo(3, 4);
}

main()

This code only uses the y parameter: 4


()

#### Ignoring Parts of a Value with a Nested `_`

In [34]:
let mut setting_value = Some(5);
let new_setting_value = Some(10);

match (setting_value, new_setting_value) {
    (Some(_), Some(_)) => { //  we don’t need to match on or use the values 
    // inside either Some variant, but we do need to test for the case when 
    // setting_value and new_setting_value are the Some variant.
        println!("Can't overwrite an existing customized value");
    }
    _ => {
        setting_value = new_setting_value;
    }
}

println!("setting is {:?}", setting_value);

Can't overwrite an existing customized value
setting is Some(5)


In [35]:
let numbers = (2, 4, 8, 16, 32);

match numbers {
    (first, _, third, _, fifth) => {
        println!("Some numbers: {first}, {third}, {fifth}")
    }
}

Some numbers: 2, 8, 32


()

#### Ignoring an Unused Variable by Starting Its Name with `_`


In [38]:
fn main() {
    let _x = 5;
    let y = 10;
} // Compiling this code will give a warning about not using the variable y, 
  // but we won’t get a warning about not using _x.

<span style="color:lightgreen">*Difference between `_` and `_x`: The syntax `_x` still binds the value to the variable, whereas `_` doesn’t bind at all*</span>

For example, below we bound a string to `_s`, which is an unused variable starting with an underscore but still binds the value and might take ownership of the value

In [39]:
let s = Some(String::from("Hello!"));

if let Some(_s) = s {
    println!("found a string");
}

println!("{:?}", s);

Error: unused variable: `y`

Error: borrow of partially moved value: `s`

Using only `_`, the value will not be bound

In [40]:
let s = Some(String::from("Hello!"));

if let Some(_) = s {
    println!("found a string");
}

println!("{:?}", s);

found a string
Some("Hello!")


#### Ignoring Remaining Parts of a Value with `..`

In [41]:
fn main() {
    struct Point {
        x: i32,
        y: i32,
        z: i32,
    }

    let origin = Point { x: 0, y: 0, z: 0 };

    match origin {
        Point { x, .. } => println!("x is {}", x),
    }
}

main()

x is 0


()

In [42]:
fn main() {
    let numbers = (2, 4, 8, 16, 32);

    match numbers {
        (first, .., last) => {
            println!("Some numbers: {first}, {last}");
        }
    }
}

main()

Some numbers: 2, 32


()

### Extra Conditionals with Match Guards

*A match guard is an additional `if` condition, specified after the pattern in a match arm, that must also match for that arm to be chosen*. Match guards are useful for expressing more complex ideas than a pattern alone allows.

In [46]:
fn main() {
    let num = Some(4);

    match num {
        Some(x) if x % 2 == 0 => println!("The number {} is even", x),
        Some(x) => println!("The number {} is odd", x),
        None => (),
    }
}

main()

The number 4 is even


()

 <span style="color:orange">*The downside of this additional expressiveness is that the compiler doesn't try to check for exhaustiveness when match guard expressions are involved.*</span>

Also, we can use a match guard to test for equality with an outer variable:

In [48]:
fn main() {
    let x = Some(10);
    let y = 10;

    match x {
        Some(50) => println!("Got 50"),
        Some(n) if n == y => println!("Matched, n = {n}"),  // The match guard if n == y is not a pattern 
                                        // and therefore doesn’t introduce new variables.
                                        // This y is the outer y rather than a new shadowed y
        _ => println!("Default case, x = {:?}", x),
    }

    println!("at the end: x = {:?}, y = {y}", x);

}
main()

Matched, n = 10
at the end: x = Some(10), y = 10


()

You can also use the or operator `|` in a match guard to specify multiple patterns

In [49]:
fn main() {
    let x = 4;
    let y = false;

    match x {
        4 | 5 | 6 if y => println!("yes"),
        _ => println!("no"),
    }
}

main()

no


()

### `@` Bindings

The at operator `@` lets us create a variable that holds a value at the same time as we’re testing that value for a pattern match.

For example, below we want to test that a `Message::Hello` `id` field is within the range `3..=7`. We also want to bind the value to the variable `id_variable` so we can use it in the code associated with the arm

In [57]:
enum Message {
    Hello { id: i32 },
}

let msg = Message::Hello { id: 11 };

match msg {
    Message::Hello {
        id: id_variable @ 3..=7,  // By specifying id_variable @ before the range 3..=7, 
                                // we’re capturing whatever value matched the range 
                                // while also testing that the value matched the range pattern.
    } => println!("Found an id in range: {}", id_variable),
    Message::Hello { id: id_variable2 @ 10..=12 } => {
        println!("Found an id in another range: {}", id_variable2)
    }
    Message::Hello { id } => println!("Found some other id: {}", id),
}


Found an id in another range: 11


()